In [1]:
import sys
from pathlib import Path
project_root = Path.cwd().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))


# Генетическая стратификация

Этот ноутбук показывает, как использовать взаимозаменяемые алгоритмы **GenRest** на синтетическом наборе из 600 наблюдений. Мы зададим выраженные зависимости между цветом, формой и возрастом, преобразуем возраст в категории, а затем запустим `GeneticStratificationAlgorithm`, чтобы увидеть заметное снижение дисперсии по сравнению с «сырыми» данными.

In [2]:
import pandas as pd
import numpy as np
from genrest import (
    GeneticStratificationAlgorithm,
    InheritedGeneticStratificationAlgorithm,
    bin_numeric,
)

rng = np.random.default_rng(0)
colors = rng.choice(["ruby", "amber", "teal"], size=600, p=[0.45, 0.35, 0.2])
shapes = rng.choice(["circle", "square", "triangle"], size=600, p=[0.4, 0.4, 0.2])
age = rng.normal(42, 11, size=600)

color_effect = np.select(
    [colors == "ruby", colors == "amber"],
    [4.5, 2.0],
    default=-1.5,
)
shape_effect = np.select(
    [shapes == "circle", shapes == "square"],
    [3.0, -0.5],
    default=-2.0,
)
age_effect = np.select(
    [age < 35, age < 45, age < 55],
    [-1.0, 0.5, 2.0],
    default=3.2,
)

y = color_effect + shape_effect + age_effect + rng.normal(0, 0.4, size=600)

data = pd.DataFrame({
    "color": colors,
    "shape": shapes,
    "age": age,
    "y": y,
})

data.head()

,color,shape,age,y
0,amber,circle,53.904412,7.018824
1,ruby,square,56.275646,6.974324
2,ruby,square,47.098692,5.598500
3,ruby,square,35.296698,4.552813
4,teal,square,35.910173,-1.725747


Преобразуем числовой столбец `age` в четыре категории с помощью функции `bin_numeric` и посмотрим на результат.

In [3]:
bin_numeric(data, 'age', bins=4)
data.head()

,color,shape,age,y
0,amber,circle,"(49.913, 72.161]",7.018824
1,ruby,square,"(49.913, 72.161]",6.974324
2,ruby,square,"(42.273, 49.913]",5.598500
3,ruby,square,"(33.789, 42.273]",4.552813
4,teal,square,"(33.789, 42.273]",-1.725747


Теперь запустим генетический алгоритм. За счёт `n_groups=2` получаем 2^3=8 страт, а более длинная эволюция (`population_size=25`, `generations=40`) помогает устойчиво находить решение с минимальной дисперсией.

In [4]:
stratifier = GeneticStratificationAlgorithm(
    strat_columns=['color', 'shape', 'age'],
    target_col='y',
    population_size=25,
    generations=40,
    n_groups=2,
    random_state=0,
)
best = stratifier.fit(data)
best, stratifier.best_score_

(Stratification(boundaries={'color': {'amber': 1, 'ruby': 0, 'teal': 1}, 'shape': {'circle': 0, 'square': 1, 'triangle': 1}, 'age': {'(33.789, 42.273]': 1, '(42.273, 49.913]': 0, '(49.913, 72.161]': 1, '(7.739, 33.789]': 0}}),
 0.4990934492798214)

### Преобразование признаков в метку страты

После обучения можно заменить несколько категориальных признаков одной колонкой с описанием полученной страты.

In [5]:
collapsed = stratifier.transform(data, column_name='strata_label')
collapsed.head()


Результаты трансформации:
  Цельная дисперсия: 11.283086
  Стратифицированная дисперсия по исходным комбинациям: 0.013610
  Стратифицированная дисперсия после объединения категорий: 0.499093
  Стратифицированная дисперсия по сравнению с исходной: понижения нет (рост на 3567.24% относительно исходной стратифицированной дисперсии)


,y,strata_label
0,7.018824,"strata_5: color={amber, teal}; shape={circle};..."
1,6.974324,"strata_3: color={ruby}; shape={square, triangl..."
2,5.598500,"strata_2: color={ruby}; shape={square, triangl..."
3,4.552813,"strata_3: color={ruby}; shape={square, triangl..."
4,-1.725747,"strata_7: color={amber, teal}; shape={square, ..."


### Сравнение дисперсий

Метрики ниже подтверждают, что GenRest значительно уменьшает дисперсию относительно необработанных данных и простых одномерных страт. Сообщение `transform` сравнивает результат с исходными комбинациями всех категориальных признаков, поэтому при сильном уменьшении количества страт оно может показывать рост относительно этого ориентира.

In [6]:
from genrest.genetic_stratifier import _weighted_stratified_variance

overall_var = data['y'].var(ddof=1)
color_var = _weighted_stratified_variance(data['y'], data['color'])
shape_var = _weighted_stratified_variance(data['y'], data['shape'])
age_var = _weighted_stratified_variance(data['y'], data['age'])
baseline_var = _weighted_stratified_variance(
    data['y'],
    data[['color', 'shape', 'age']].astype(str).agg('|'.join, axis=1),
)
algo_var = stratifier.best_score_

print(f"Цельная дисперсия: {overall_var:.4f}")
print(f"Стратификация только по color: {color_var:.4f}")
print(f"Стратификация только по shape: {shape_var:.4f}")
print(f"Стратификация только по age: {age_var:.4f}")
print(f"Комбинации всех признаков: {baseline_var:.4f}")
print(f"GenRest после объединения категорий: {algo_var:.4f}")
print(f"Понижение относительно цельной дисперсии: {(1 - algo_var / overall_var) * 100:.2f}%")

Цельная дисперсия: 11.2831
Стратификация только по color: 2.3626
Стратификация только по shape: 2.4901
Стратификация только по age: 2.5067
Комбинации всех признаков: 0.0136
GenRest после объединения категорий: 0.4991
Понижение относительно цельной дисперсии: 95.58%


## Наследуемые колонки
`InheritedGeneticStratificationAlgorithm` позволяет указать признаки, которые нельзя смешивать между стратами.
Алгоритм совместим с базовым, а результаты можно комбинировать в одном пайплайне.

In [7]:
inherited_algo = InheritedGeneticStratificationAlgorithm(
    strat_columns=['color', 'shape', 'age'],
    target_col='y',
    mandatory_columns=['color'],
    n_groups=2,
    generations=40,
    population_size=25,
    random_state=0,
)
inherited_algo.fit(data)
inherited_algo.transform_to_indices(data)[:5]

array([ 0,  6,  6,  7, 11])

### Преобразование с описанием страт
После обучения можно получить датафрейм с метками, где сочетаются обязательные и объединённые признаки.

In [8]:
inherited_view = inherited_algo.transform(
    data,
    column_name='inherited_strata',
    drop_original=False,
)
inherited_view.head()


Результаты трансформации с обязательными колонками:
  Цельная дисперсия: 11.283086
  Стратифицированная дисперсия по исходным комбинациям: 0.013610
  Стратифицированная дисперсия после объединения категорий: 0.110075
  Стратифицированная дисперсия по сравнению с исходной: понижения нет (рост на 708.81% относительно исходной стратифицированной дисперсии)


,color,shape,age,y,inherited_strata
0,amber,circle,"(49.913, 72.161]",7.018824,color=amber; strata_0: shape={circle}; age={(4...
1,ruby,square,"(49.913, 72.161]",6.974324,"color=ruby; strata_6: shape={square, triangle}..."
2,ruby,square,"(42.273, 49.913]",5.598500,"color=ruby; strata_6: shape={square, triangle}..."
3,ruby,square,"(33.789, 42.273]",4.552813,"color=ruby; strata_7: shape={square, triangle}..."
4,teal,square,"(33.789, 42.273]",-1.725747,"color=teal; strata_11: shape={square, triangle..."


### Быстрый запуск через `fit_transform`
Если не нужно сохранять объект, используйте сокращённый вызов, который сразу возвращает преобразованные данные.

In [9]:
quick_view = InheritedGeneticStratificationAlgorithm(
    strat_columns=['color', 'shape', 'age'],
    target_col='y',
    mandatory_columns=['color'],
    n_groups=2,
    generations=40,
    population_size=25,
    random_state=1,
).fit_transform(
    data,
    column_name='quick_strata',
    drop_original=True,
)
quick_view.head()


Результаты трансформации с обязательными колонками:
  Цельная дисперсия: 11.283086
  Стратифицированная дисперсия по исходным комбинациям: 0.013610
  Стратифицированная дисперсия после объединения категорий: 0.110075
  Стратифицированная дисперсия по сравнению с исходной: понижения нет (рост на 708.81% относительно исходной стратифицированной дисперсии)


,y,quick_strata
0,7.018824,color=amber; strata_0: shape={circle}; age={(4...
1,6.974324,"color=ruby; strata_6: shape={square, triangle}..."
2,5.598500,"color=ruby; strata_6: shape={square, triangle}..."
3,4.552813,"color=ruby; strata_7: shape={square, triangle}..."
4,-1.725747,"color=teal; strata_11: shape={square, triangle..."


## Стратификация по числовому признаку
`bin_numeric` превращает вещественные данные в категории перед запуском алгоритма.


In [10]:
df = pd.DataFrame({'x': rng.normal(size=20), 'y': rng.normal(size=20)})
bin_numeric(df, 'x', bins=4)
GeneticStratificationAlgorithm(['x'], 'y', population_size=5, generations=5, n_groups=2, random_state=0).fit(df)

Stratification(boundaries={'x': {'(-0.657, 0.335]': 1, '(-1.7109999999999999, -0.657]': 1, '(0.335, 0.67]': 1, '(0.67, 2.379]': 0}})